In [1]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib import colors
from math import sqrt
from math import log

# Move class for Breakthrough

In [2]:
class Move(object):
    def __init__(self, color, x1, y1, x2, y2):
        self.color = color
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        
    def valid (self, board):
        if self.x2 >= Dx or self.y2 >= Dy or self.x2 < 0 or self.y2 < 0:
            return False
        if self.color == White:
            if self.x2 != self.x1 + 1:
                return False
            if board.board [self.x2] [self.y2] == Black:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1:
                    return True
                return False
            elif board.board [self.x2] [self.y2] == Empty:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1 or self.y2 == self.y1:
                    return True
                return False
        elif self.color == Black:
            if self.x2 != self.x1 - 1:
                return False
            if board.board [self.x2] [self.y2] == White:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1:
                    return True
                return False
            elif board.board [self.x2] [self.y2] == Empty:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1 or self.y2 == self.y1:
                    return True
                return False
        return False
    
    def code (self, board):
        direction = 0
        if self.y2 > self.y1:
            if board.board [self.x2] [self.y2] == Empty:
                direction = 1
            else: 
                direction = 2
        if self.y2 < self.y1:
            if board.board [self.x2] [self.y2] == Empty:
                direction = 3
            else:
                direction = 4
        if self.color == White:
            return 5 * (Dy * self.x1 + self.y1) + direction
        else:
            return 5 * Dx * Dy + 5 * (Dy * self.x1 + self.y1) + direction


        

# Board class to play Breakthrough 5x5

In [3]:
import random

Dx = 5
Dy = 5
Empty = 0
White = 1
Black = 2

class Board(object):
    def __init__(self):
        self.h = 0
        self.turn = White
        self.board = np.zeros ((Dx, Dy))
        for i in range (0, 2):
            for j in range (0, Dy):
                self.board [i] [j] = White
        for i in range (Dx - 2, Dx):
            for j in range (0, Dy):
                self.board [i] [j] = Black
    
    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                if self.board [i] [j] == self.turn:
                    for k in [-1, 0, 1]:
                        for l in [-1, 0, 1]:
                            m = Move (self.turn, i, j, i + k, j + l)
                            if m.valid (self):
                                moves.append (m)
        return moves
    
    def score (self):
        for i in range (0, Dy):
            if (self.board [Dx - 1] [i] == White):
                return 1.0
            elif (self.board [0] [i] == Black):
                return 0.0
        l = self.legalMoves ()
        if len (l) == 0:
            if self.turn == Black:
                return 1.0
            else:
                return 0.0
        return 0.5

    def terminal (self):
        if self.score () == 0.5:
            return False
        return True
    
    def play (self, move):
        self.board [move.x1] [move.y1] = Empty
        self.board [move.x2] [move.y2] = move.color
        if (self.turn == White):
            self.turn = Black
        else:
            self.turn = White

    def playout (self):
        while (True):
            moves = self.legalMoves ()
            if self.terminal ():
                return self.score ()
            n = random.randint (0, len (moves) - 1)
            self.play (moves [n])

    def print(self):
        print("   1 2 3 4 5")
        for i in range(Dy):
            print("{} |".format(i + 1), end="")
            for j in range(Dx):
                if self.board [i] [j] == Black:
                    print("\u265F", end="")
                elif self.board [i] [j] == White:
                    print("\u2659", end="")
                else:
                    print(" ", end="")
                if j < Dx:
                    print("|", end="")

            if i < Dy:
                print()        


# Flat Monte Carlo

In [4]:
import copy
def flat (board, n):
    moves = board.legalMoves ()
    bestScore = 0
    bestMove = 0
    for m in range (len(moves)):
        sum = 0
        for i in range (n // len (moves)):
            b = copy.deepcopy (board)
            b.play (moves [m])
            r = b.playout ()
            if board.turn == Black:
                r = 1 - r
            sum = sum + r
        if sum > bestScore:
            bestScore = sum
            bestMove = m
    return moves [bestMove]

# UCB

# Board class with hashcode

In [5]:
import random

Dx = 5
Dy = 5
Empty = 0
White = 1
Black = 2

hashTable = []
for k in range (3):
    l = []
    for i in range (Dx):
        l1 = []
        for j in range (Dy):
            l1.append (random.randint (0, 2 ** 64))
        l.append (l1)
    hashTable.append (l)
hashTurn = random.randint (0, 2 ** 64)

class Board(object):
    def __init__(self):
        self.h = 0
        self.turn = White
        self.board = np.zeros ((Dx, Dy))
        for i in range (0, 2):
            for j in range (0, Dy):
                self.board [i] [j] = White
        for i in range (Dx - 2, Dx):
            for j in range (0, Dy):
                self.board [i] [j] = Black
    
    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                if self.board [i] [j] == self.turn:
                    for k in [-1, 0, 1]:
                        for l in [-1, 0, 1]:
                            m = Move (self.turn, i, j, i + k, j + l)
                            if m.valid (self):
                                moves.append (m)
        return moves
    
    def score(self):
        for i in range (0, Dy):
            if (self.board [Dx - 1] [i] == White):
                return 1.0
            elif (self.board [0] [i] == Black):
                return 0.0
        l = self.legalMoves ()
        if len (l) == 0:
            if self.turn == Black:
                return 1.0
            else:
                return 0.0
        return 0.5

    def terminal(self):
        if self.score () == 0.5:
            return False
        return True
    
    def playout(self):
        while (True):
            moves = self.legalMoves ()
            if self.terminal ():
                return self.score ()
            n = random.randint (0, len (moves) - 1)
            self.play (moves [n])
            
    def play(self, move):
        col = int (self.board [move.x2] [move.y2])
        if col != Empty:
            self.h = self.h ^ hashTable [col] [move.x2] [move.y2]
        self.h = self.h ^ hashTable [move.color] [move.x2] [move.y2]
        self.h = self.h ^ hashTable [move.color] [move.x1] [move.y1]
        self.h = self.h ^ hashTurn
        self.board [move.x2] [move.y2] = move.color
        self.board [move.x1] [move.y1] = Empty
        if (move.color == White):
            self.turn = Black
        else:
            self.turn = White
            
    def print(self):
        print("   1 2 3 4 5")
        for i in range(Dy):
            print("{} |".format(i + 1), end="")
            for j in range(Dx):
                if self.board [i] [j] == Black:
                    print("\u265F", end="")
                elif self.board [i] [j] == White:
                    print("\u2659", end="")
                else:
                    print(" ", end="")
                if j < Dx:
                    print("|", end="")

            if i < Dy:
                print()


# Transposition Table

In [6]:
MaxLegalMoves = 6 * Dx
Table = {}

def add (board):
    nplayouts = [0.0 for x in range (MaxLegalMoves)]
    nwins = [0.0 for x in range (MaxLegalMoves)]
    Table [board.h] = [0, nplayouts, nwins]

def look (board):
    return Table.get (board.h, None)


# UCT

# Tournament between Flat, UCB and UCT